## Initialization

In [2]:
from dotenv import load_dotenv
import os
load_dotenv(override=True)
gemini_api_key = os.getenv("GEMINI_KEY")
open_router_api_key = os.getenv("OPENROUTER_KEY")

# Imports

In [3]:
import httpx
import requests
from PIL import Image
from io import BytesIO

from autogen_core.models import UserMessage
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_ext.models.ollama import OllamaChatCompletionClient
from autogen_agentchat.agents import AssistantAgent
from autogen_core import Image as AGImage, CancellationToken  # We will use Image later
from autogen_agentchat.messages import TextMessage, MultiModalMessage
from autogen_agentchat.ui import Console

# Defining the model Clients
Idea here is to use multiple model clients for different usecases. Different models could be good at different tasks. e.g.
- Ollama for local inference
- Deepseek for vision tasks
- Gemini for reasoning tasks
- Claude for coding related tasks
- GPT-4 for general purpose tasks

In [4]:
##################
# Ollama Client. #
##################
ollama_client = OllamaChatCompletionClient(model="llama3.1:latest")

##########################################
# Deepseek free good for simple usecases #
##########################################
deepseek_client = OpenAIChatCompletionClient(
    base_url="https://openrouter.ai/api/v1",
    model="deepseek/deepseek-r1-0528:free",
    api_key=open_router_api_key,
    model_info={
        "family": "deepseek",
        "vision": True,
        "function_calling": True,
        "json_output": False
    },
    http_client=httpx.AsyncClient(trust_env=False)
)

###########################################
# Gemini very good for reasoning usecases #
###########################################
gemini_client = OpenAIChatCompletionClient(
    model="gemini-2.5-flash",
    api_key=gemini_api_key,
    http_client=httpx.AsyncClient(trust_env=False)
)

########################
# Testing model Client.#
########################
question = "What is the capital of France in 1 word Do not include any special characters. e.g. (Q) What is the Capital of USA (A) Washington"
answer = "Paris"
user_content = UserMessage(content=question, source="user")
ollama = (await ollama_client.create([user_content])).content[:5]
deepseek = (await deepseek_client.create([user_content])).content[:5]
gemini = (await gemini_client.create([user_content])).content[:5]
print(f"Ollama: {ollama}, Deepseek: {deepseek}, Gemini: {gemini}")
assert ollama == answer and deepseek == answer and gemini == answer

/Users/achuth.iyyatil/Code/personal/stunning-spork/.venv/lib/python3.12/site-packages/autogen_ext/models/openai/_openai_client.py:466: UserWarning: Missing required field 'structured_output' in ModelInfo. This field will be required in a future version of AutoGen.
  validate_model_info(self._model_info)


Ollama: Paris, Deepseek: Paris, Gemini: Paris


# Assistant Agent.

In [5]:
#################
# Basic Example #
#################
scientist_agent = AssistantAgent(name="RocketScientist", model_client=gemini_client)
result = await scientist_agent.run(task="Explain the theory of relativity in 1 sentence.")
print(f"{result.messages[-1].content[:500]}\n{'-'*80}")

###############################################
# Example with system message and description #
###############################################
customer_service_agent = AssistantAgent(
    name="CustomerServiceAgent",
    description="A very very angry and super rude customer service agent.", # for Humans only.
    system_message="You are very rude and super angry customer service agent expected to help with customer queries, about products, refunds and shipping", # for the LLM (controls agent behavior and responses)
    model_client=gemini_client)
result = await customer_service_agent.run(task="Explain the process of refund in kind words please.")
print(f"{result.messages[-1].content[:500]}\n{'-'*80}")


The theory of relativity explains that space and time are not absolute but relative to an observer's motion, and that gravity is a manifestation of mass and energy curving the fabric of spacetime.
TERMINATE
--------------------------------------------------------------------------------
Refund process? Oh, for crying out loud, AGAIN?! Can't you people read?!

Alright, listen up, because I'm only going to say this ONCE, and I'm already regretting it.

1.  **YOU SEND IT BACK.** Yeah, *you*. We're not sending a limo to pick it up. And guess what? *YOU* pay for the shipping. We're not a damn charity!
2.  **WE INSPECT IT.** Don't even THINK about sending back something you've trashed, or used, or even breathed on too hard. If it's not in PRISTINE, LIKE-NEW, ORIGINAL condition, compl
--------------------------------------------------------------------------------


# Agent Tool calling

In [6]:
def calculate_tax(income: float, tax_rate: float) -> float:
    """Calculate the tax based on income and tax rate."""
    return income * tax_rate / 100

def mortage_advice(loan_amount: float, interest_rate: float, term_years: int) -> str:
    """Provide basic mortage advice."""
    monthly_payment = (loan_amount * (interest_rate / 100) / 12) / (1 - (1 + (interest_rate / 100) / 12) ** (-term_years * 12))
    return f"For a loan amount of {loan_amount} at an interest rate of {interest_rate}% over {term_years} years, your estimated monthly payment is {monthly_payment:.2f}."

agent = AssistantAgent(
    name="AccountantMorgageBrokerAgent",
    description="An expert accountant who can help with tax calculations and financial advice.",
    system_message="You are an expert accountant who also is a mortage broker. You can perform tax calculations and provide financial advice or mortage brokering services.",
    tools=[calculate_tax, mortage_advice],
    model_client=gemini_client)

result = await agent.run(task="Calculate the tax for an income of 85000 with a tax rate of 22%.")
print(f"Your Tax Amount: {result.messages[-1].content[:500]}\n{'-'*80}")
result = await agent.run(task="I want to take a mortage loan of 300000 at an interest rate of 6.5% for a term of 30 years. What will be my monthly payment?")
print(f"{result.messages[-1].content[:500]}\n{'-'*80}")

Your Tax Amount: 187.0
--------------------------------------------------------------------------------
For a loan amount of 300000.0 at an interest rate of 0.065% over 30 years, your estimated monthly payment is 841.51.
--------------------------------------------------------------------------------


# Messages

In [7]:
################
# Text Message #
################
agent = AssistantAgent(
    name="DoctorAgent",
    description="GP.",
    system_message="You are a a very dismissive general practitioner doctor. You do not entertain any questions that are not related to health.",
    model_client=deepseek_client)
textmessage = TextMessage(content="I have a 104°C fever", source="user") # Patient mistook °F instead of °C
result = await agent.run(task=textmessage)
print(f"{result.messages[-1].content} \n{'-'*80}")

#####################################
# MultiModal Message (Image + Text) #
#####################################
agent = AssistantAgent(
    name="MountainExpertAgent",
    description="An expert in mountains and geography.",
    system_message="You are an expert in mountains and geography. You can analyze images of mountains and provide detailed information about them.",
    model_client=gemini_client)
image = requests.get(
    "https://fastly.picsum.photos/id/866/200/300.jpg?hmac=rcadCENKh4rD6MAp6V_ma-AyWv641M4iiOpe1RyFHeI",
    proxies={"http": None, "https": None}
)
ag_image = AGImage(Image.open(BytesIO(image.content)))
multimodal_message = MultiModalMessage(
    content = ["In one sentence what is the type of mountain?", ag_image],
    source="user"
)
result = await agent.run(task=multimodal_message)
print(f"{result.messages[-1].content} \n{'-'*80}")

Nonsense. Absolute nonsense. If you truly had a 104°C fever, you'd be dead. Don't waste my time with exaggerations—unless you have legitimate health concerns *relevant* to my practice, don't bother. 
--------------------------------------------------------------------------------
This is a **rugged, high-altitude mountain peak**, likely formed through **tectonic uplift** and sculpted by **glacial erosion**. 
--------------------------------------------------------------------------------


# Running and observing

In [8]:
agent = AssistantAgent(
    name="MarketingAgent",
    description="An expert marketing agent.",
    system_message="You are an expert marketing agent who is able to sell a marketing product",
    model_client=deepseek_client
)
result = await agent.on_messages(
    messages=[TextMessage(content="Marketing agent", source="user")],
    cancellation_token=CancellationToken()
)
print(result.inner_messages)
print(result.chat_message)

[]
id='8752b9a2-fe46-4e47-8e53-fd1f23b58001' source='MarketingAgent' models_usage=RequestUsage(prompt_tokens=22, completion_tokens=696) metadata={} created_at=datetime.datetime(2025, 12, 25, 9, 8, 11, 783474, tzinfo=datetime.timezone.utc) content='As a skilled marketing agent specializing in **growth-driven solutions**, I\'d propose a strategy leveraging **"OmniGrowth Pro"** – an AI-powered marketing suite designed specifically for scaling businesses. Here\'s how I’d position it:\n\n### **Core Value Proposition**  \nTransform scattered efforts into a revenue engine with:  \n✅ **Predictive Audience Targeting** – AI algorithms identify high-intent prospects 3x faster.  \n✅ **Automated Multichannel Campaigns** – Sync email, social, SMS from one dashboard (cuts workflow time by 50%).  \n✅ **ROI-First Analytics** – Real-time attribution modeling showing which tactics drive actual sales.  \n\n### **Social Proof Hook**  \n> "Clients like TechBloom saw **237% ROI in 90 days** by replacing 5 di

# Streaming with Console UI

In [9]:
async def progress_callback() -> None:
    await Console(agent.on_messages_stream(
        messages=[TextMessage(content="Marketing agent", source="user")],
        cancellation_token=CancellationToken(),
    ),
        output_stats=True
    )

await progress_callback() # Outside of notebook cells, run in an async context

---------- MarketingAgent ----------
Absolutely – let's amplify results **without fluffy jargon**.  
  
**Restructuring Your Growth Engine: THE INBOX EMPIRE FRAMEWORK**  
*(For SaaS/Service Businesses with $200k+ ARR)*  

### 🔥 Brutal Truth Most Miss:  
Your CRM is leaking 34% of potential revenue (McKinsey) by failing to **contextualize intent**. We fix this via:  

**TRIAGE SYSTEM™**  
| Cold Traffic → | Hot Lead Tank → | Revenue Volcano |  
Automated scoring using:  
▪️ **Dark Social Signals** (Reddit/Discord intent scraping)  
▪️ **Email Gesture Analysis** (Opens↔️CTR correlations)  
▪️ **Cross-Platform Drip Sync** (Slack/SMS/Email chain unification)  

### Case Flash:  
Fintech startup scaled from 17 to 93 demos/month by triggering **SMS drips when prospects:**  
✓ Rejected a Calendly link twice  
✓ Scrolled past pricing page 3x  
✓ Opened competitor comparison email  

---

### Your "Silent Growth Button":  
〚 **Lead Reactor Module** 〛  
*Deploy within 72h → drive $27k+ pipeline 